In [ ]:
#Loading packages
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
import re
#import scrape_functions_un as kzd
import scraping_model as sm 

## Players Dictionary

In [ ]:
players_names = {'Rory McIlroy': '17419',
            'Tiger Woods': '10085', 
            'Jordan Spieth': '19976',
            'Justin Thomas': '20037',
            'Scottie Scheffler': '25786',
            'Collin Morikawa': '33992',
            'Sergio García': '10170',
            'Dustin Johnson': '17976',
            'Phil Mickelson': '10083',
            'Ian Poulter': '11956',
            'Talor Gooch': '27168',
            'Cameron Smith': '27755',
            'Joaquin Niemann': '33997',
            'Bryson DeChambeau': '29148',
            'Brooks Koepka': '25730',
            'Jon Rahm': '28177'}

# Posible other players:
# Bubba Watson, Viktor Hovland, Ricky Fowler, Matt Fitzpatrick, Tommy Fleetwood
# Tony Finau, Jason Day, Abraham Ancer, Paul Casey, Tyrrell Hatton, Justin Rose
# Kevin Na, Louis Oosthuizen, Ptrick Reed, Lee Westwood, Xander Schauffele, Max Homa

### We want to get all the dates and urls for each of the player's pages

### Doing a for loop, to get all the links for each player

In [ ]:
links = {}
for player, id in players_names.items():
    player_url = f'https://www.asapsports.com/show_player.php?id={id}'
    response = requests.get(player_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dates_links = soup.find_all('b')
    links_player = []

    for row in dates_links:
            a_tags = soup.find_all('a', href=True)
        
    for atag in a_tags:
        url = atag['href']
        if url.startswith('http://www.asapsports.com/show_interview.php?id='):
            links_player.append(url)
    links[f'{player}'] = links_player
    print('URLs fetched from:', player)

# #3m 30.3s    

In [ ]:
# Flatten the dictionary
data = []
for player, links_ls in links.items():
    for link in links_ls:
        data.append({"Player": player, "Link": link})
        
players_links_df = pd.DataFrame(data)
players_links_df.to_csv('Players Links', sep=',', index=False, encoding='utf-8')

In [20]:
players_links_df = pd.read_csv('Players Links')

## Big loop, for all the links for all the players

In [47]:
press_conferences = pd.DataFrame(columns=['date','question','answer'])
links = list(players_links_df['Link'])
trouble_links = []

#### Agregar al file de funciones

In [48]:
from datetime import datetime


def relevant_conferences(date_string):
    date_obj = datetime.strptime(date_string, "%B %d, %Y")
    comparison_date = datetime(2020, 10, 30)
    if date_obj > comparison_date:
        return True
    else:
        return False

In [49]:
for link in links:
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:121.0) Gecko/20100101 Firefox/121.0'}
    response = requests.get(link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Getting Data
    event = soup.find('h1').text
    player = soup.find('h3').text
    date = soup.find('h2').text

    if relevant_conferences(date) == True:
        try:

            dates_loop, answers_loop = sm.parse_interview(link)

            bold_text = soup.find_all('b')

            questions = sm.get_questions(bold_text) 
            stripped_questions = [question.strip('Q.') for question in questions]
            stripped_questions = [sm.clean_string(question).split() for question in stripped_questions]

            num_to_fill = len(answers_loop) - len(stripped_questions)
            stripped_questions = ["MODERATOR"] * num_to_fill + stripped_questions

            temp_df = pd.DataFrame({'date': dates_loop, 'answer': answers_loop, 'question': stripped_questions})
            press_conferences = pd.concat([press_conferences, temp_df], ignore_index=True)
            print('Scrapped press conference from:',event, date,'\t by:',player)
        except:
            print(f"An error occurred while processing {event}, {date}\t by:{player}")
            trouble_links.append(link)
    else:
        print('Before Masters 2020',player)
        pass

# 2 hrs    

Scrapped press conference from: COGNIZANT CLASSIC March 3, 2024 	 by: Rory McIlroy
Scrapped press conference from: COGNIZANT CLASSIC February 29, 2024 	 by: Rory McIlroy
Scrapped press conference from: COGNIZANT CLASSIC February 28, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 21, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 21, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 21, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 20, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 17, 2024 	 by: Rory McIlroy
Scrapped press conference from: HERO DUBAI DESERT CLASSIC January 17, 2024 	 by: Rory McIlroy
Scrapped press conference from: DUBAI INVITATIONAL January 14, 2024 	 by: Rory McIlroy
Scrapped press conference from: DUBAI INVITATIONAL January 13, 2024 	 by: Rory McIlroy
Scrap

since the moderator questions don't have a fixed pattern, we struggle to retrieve them, we will impute them with 'moderator' in the question; in the text analysis we might get rid of the token so it doesn't consider it an important token.

In [51]:
press_conferences.to_csv('Press_Conferences', sep=',', index=False, encoding='utf-8')

,date,question,answer
0,"March 3, 2024","[a, little, bit, of, a, race, to, the, finish,...","(rory mcilroy, [a, week, that, could, have, be..."
1,"March 3, 2024","[you, added, this, event, added, valero, to, b...","(rory mcilroy, [yeah, if, anything, i'm, just,..."
2,"March 3, 2024","[unless, you, really, think, about, your, week...","(rory mcilroy, [i'm, really, happy, i, got, do..."
3,"March 3, 2024","[with, shane, kind, of, in, contention, you're...","(rory mcilroy, [it, would, mean, a, lot, he, l..."
4,"February 29, 2024","[what, did, you, think, of, your, round, today]","(rory mcilroy, [pretty, solid, some, good, som..."
...,...,...,...
19347,"November 10, 2020",MODERATOR,"(jon rahm, [it, could, go, two, ways, right, s..."
19348,"November 10, 2020",MODERATOR,"(jon rahm, [yeah, on, i, hit, it, to, about, a..."
19349,"November 10, 2020",MODERATOR,"(jon rahm, [nothing, special, what, can, we, d..."
19350,"November 10, 2020",MODERATOR,"(jon rahm, [that's, what, we, said, it, would,..."


### Export final dataframe into CSV for use